# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [72]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [73]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L,U = diy_lu(a)

np.linalg.matrix_rank(a)

6

In [74]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)


In [75]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [76]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [77]:
np.linalg.matrix_rank(a1)

6

In [78]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\polina\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\polina\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [79]:
def minor(c):
    i=1
    flag = True
    while i!=N+1 and flag == True:
        b = c[:i,:i].copy()
        if np.linalg.det(b)==0:
            flag=False
        i+=1
    return flag

print(minor(a), minor(a1))


True False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [82]:
# ... ENTER YOUR CODE HERE ...
def diy_lu_mod(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):

        P_j = np.eye(N)
        m = np.argmax(a[j:, j])
        P_j[m], P_j[j] = P_j[j].copy(), P_j[m].copy()
        P = P @ P_j
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        lam = lam @ P_j
        
        u = lam  @ u

        lam[j+1:, j] = gamma
        lam = lam @ P_j 
        L = L @ lam
    return L, u, P
L,u,P = diy_lu_mod(a)
print("matrix a", P@L@u)
print(L, u)
print("matrix a1")
L1, u1, P1 = diy_lu_mod(a1)
print(L1, u1)

matrix a [[  2.051   3.106   3.9     4.363   4.663   4.872]
 [ -6.694  -5.229  -3.79   -2.946  -2.4    -2.019]
 [-13.18   -9.433  -6.249  -4.35   -3.107  -2.233]
 [-11.041  -2.996   2.477   5.659   7.719   9.158]
 [ -0.949   0.106   0.9     1.363   1.663   1.872]
 [  2.581  15.207  22.134  25.986  28.428  30.112]]
[[ 1.     0.     0.     0.     0.     0.   ]
 [ 1.     1.     0.     0.     0.     0.   ]
 [ 2.455 -1.455  1.     0.     0.     0.   ]
 [ 3.87  -1.714 -1.156  1.     0.     0.   ]
 [ 5.213 -1.882 -1.254 -1.077  1.     0.   ]
 [ 6.512 -2.    -1.322 -1.129 -1.06   1.   ]] [[-0.949  0.106  0.9    1.363  1.663  1.872]
 [ 3.     3.     3.     3.     3.     3.   ]
 [ 0.    -1.125 -1.636 -1.929 -2.118 -2.25 ]
 [-4.364 -6.    -6.481 -6.711 -6.846 -6.935]
 [-5.143 -5.771 -5.599 -5.445 -5.33  -5.242]
 [ 4.382  6.134  6.85   7.204  7.415  7.554]]
matrix a1
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan na

C:\Users\polina\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


In [81]:
#Так как матрица а1 сингулярная и симметричная, а ее максимальный элемент как раз зануляет ее минор, ни 
#ни colomn, ни full разложения не работают